Abhängigkeiten installieren und Packages laden

In [1]:
!pip install pandas
!pip install gensim
!pip install nltk
!pip install regex
!pip install numpy
!pip install scikit-learn
!pip install pyLDAvis
!pip install matplotlib

!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install --upgrade gensim

import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models
import pyLDAvis

# wenn es nicht läuft runtime neu starten

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  A

Subsets zusammenführen, Dopplungen entfernen

In [ ]:
csv_data_raw = [r'/content/4_1_ergebnisse.csv', r'/content/4_2_ergebnisse.csv', r'/content/4_3_ergebnisse.csv', r'/content/4_4_ergebnisse.csv']

csv_pfad = '/content/05_csv_data_raw.csv' # nicht wundern, hatte datei zwischenzeitlich umbenannt

csv_no_doubles_pfad = '/content/06_data_raw_no_doubles.csv'

df = pd.concat((pd.read_csv(f) for f in csv_data_raw), ignore_index=True)

df.to_csv('csv_data_raw.csv', index=False)

def entferne_doppelte_rezensionen(csv_pfad, anzahl_rezensionen=5):
    df = pd.read_csv(csv_pfad)

    # Doppelte Rezensionen basierend auf place_id und rezension entfernen
    df = df.drop_duplicates(subset=['place_id', 'rezension'], keep='first')

    # Anzahl der Rezensionen pro Ort begrenzen
    df = df.groupby('place_id').head(anzahl_rezensionen).reset_index(drop=True)

    return df

# Funktionsaufruf
ergebnis_df = entferne_doppelte_rezensionen(csv_pfad)

# Das bereinigte DataFrame anzeigen (optional)
print(ergebnis_df)

# Das bereinigte DataFrame in eine neue CSV-Datei speichern (optional)
ergebnis_df.to_csv("06_data_raw_no_doubles.csv", index=False)

Validierung der automatischen Abrufe: Zufallsstichprobe & manueller Abgleich

In [ ]:
df = pd.read_csv(csv_no_doubles_pfad) #siehe oben

def zufallsstichprobe(csv_pfad, stichprobengroesse=100, zufalls_seed=42): #seed 42
    df = pd.read_csv(csv_no_doubles_pfad)

    # Zufallsstichprobe ziehen
    stichprobe_df = df.sample(n=min(stichprobengroesse, len(df)), random_state=zufalls_seed)

    return stichprobe_df

# Beispielaufruf:
csv_datei = csv_no_doubles_pfad # Ersetzen Sie dies durch den tatsächlichen Dateinamen
stichprobe = zufallsstichprobe(csv_datei, stichprobengroesse=100, zufalls_seed=42)

# Die Stichprobe in eine neue CSV-Datei speichern (optional):
stichprobe.to_csv("07_stichprobe_validierung.csv", index=False)

# Validierung manuell durchgeführt, für Kriterien siehe Hausarbeit;
# Kodierung: 1 = valid; 0 = not valid or valid with restrictions;


########## hier noch die Ergebnisse ausrechnen ###############

                                                     ort  \
2507   Alexius/Josef Krankenhaus, Montanusstraße 54, ...   
11482  Sophien- und Hufeland-Klinikum gGmbH, Henry-va...   
4207   Ev. Krankenhaus Lippstadt, Wiedenbrücker Straß...   
1935   Rotes-Kreuz-Krankenhaus Bremen gGmbH, St.-Paul...   
7768   Klinik Dr. Wilhelm, Hans-Urmiller-Ring 46, Wol...   
...                                                  ...   
10769  Hauptstandort Rudolf-Virchow-Klinikum, Virchow...   
10563  Sana HANSE-Klinikum Wismar GmbH, Dr.-Unruh-Str...   
8830   Bezirksklinikum Ansbach, Paracelsusstraße 36, ...   
5006   Helios Kliniken Kassel, Hansteinstraße 29, Kassel   
4216       LWL-Klinik Lippstadt, Im Hofholz 6, Lippstadt   

                                        name                    strasze  \
2507               Alexius/Josef Krankenhaus             Montanusstraße   
11482   Sophien- und Hufeland-Klinikum gGmbH  Henry-van-de-Velde-Straße   
4207               Ev. Krankenhaus Lippstadt       Wie

n_components bzw k festlegen

In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

# Daten laden und preprocessen (siehe vorherige Beispiele)

# Define Parameter Grid
param_grid = {'n_components': [2, 5, 10, 15]}

# LDA Model
lda = LatentDirichletAllocation(random_state=42)

# Grid Search
grid_search = GridSearchCV(lda, param_grid, scoring='neg_log_perplexity', cv=3)
grid_search.fit(X)

# Bestes Modell
best_lda = grid_search.best_estimator_
print("Beste Anzahl an Topics:", grid_search.best_params_)

NameError: name 'X' is not defined

LDA Analysis und erste Visualisierung

In [8]:
!pip install gensim # LDA
!pip install pyLDAvis # Visualisierung

import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models  # Wichtig: gensim_models importieren
import pyLDAvis

nltk.download('stopwords')

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', str(text))
    text = text.lower()
    tokens = text.split()
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens

def prepare_corpus(processed_docs):
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    return dictionary, corpus

def train_lda(corpus, dictionary, num_topics=5):
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42)
    return lda_model

def evaluate_model(lda_model, corpus, dictionary, processed_docs):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence)
    return coherence

def display_topics(lda_model, num_words=10):
    for idx, topic in lda_model.print_topics(-1, num_words=num_words):
        print(f'Topic: {idx} \nWords: {topic}')

# Load data
df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')
review_column = 'rezension'
if review_column not in df.columns:
    raise ValueError(f"Column '{review_column}' not found in DataFrame.")

# Preprocess data
processed_docs = df[review_column].astype(str).map(preprocess_text)

# Prepare corpus
dictionary, corpus = prepare_corpus(processed_docs)

# Train LDA model
num_topics = 5  # Adjust as needed
lda_model = train_lda(corpus, dictionary, num_topics)

# Evaluate model
coherence_score = evaluate_model(lda_model, corpus, dictionary, processed_docs)

# Display topics
display_topics(lda_model)

# Visualisierung mit pyLDAvis
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary) # prepare aus gensim_models
pyLDAvis.display(vis)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Coherence Score:  0.3598509036866703
Topic: 0 
Words: 0.021*"gut" + 0.013*"dank" + 0.013*"freundlich" + 0.012*"team" + 0.011*"station" + 0.010*"wurde" + 0.009*"ärzte" + 0.009*"klinik" + 0.009*"super" + 0.009*"vielen"
Topic: 1 
Words: 0.019*"wurde" + 0.007*"krankenhaus" + 0.007*"mal" + 0.006*"patienten" + 0.006*"arzt" + 0.006*"mehr" + 0.006*"notaufnahme" + 0.005*"leider" + 0.005*"gut" + 0.005*"schon"
Topic: 2 
Words: 0.021*"klinik" + 0.008*"wurde" + 0.007*"essen" + 0.006*"gut" + 0.006*"gibt" + 0.005*"immer" + 0.004*"freundlich" + 0.004*"zimmer" + 0.004*"schon" + 0.004*"wochen"
Topic: 3 
Words: 0.010*"wurde" + 0.009*"krankenhaus" + 0.006*"notaufnahme" + 0.006*"freundlich" + 0.006*"arzt" + 0.006*"klinik" + 0.005*"mal" + 0.005*"schmerzen" + 0.005*"wirklich" + 0.004*"super"
Topic: 4 
Words: 0.013*"klinik" + 0.012*"patienten" + 0.009*"immer" + 0.006*"gut" + 0.005*"therapeuten" + 0.005*"mal" + 0.005*"wurde" + 0.005*"personal" + 0.005*"zeit" + 0.004*"mehr"


ÜBERARBEITET:

In [ ]:
nltk.download('stopwords')


def preprocess_text(text):
    """Bereinigt den Text, entfernt Sonderzeichen und Stoppwörter."""
    text = re.sub(r'[^\w\s]', '', str(text))
    text = text.lower()
    tokens = text.split()
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens


def prepare_corpus(processed_docs):
    """Erstellt ein Gensim-Dictionary und Corpus."""
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    return dictionary, corpus


def train_lda(corpus, dictionary, num_topics=5, passes=15, iterations=400, eval_every=None):
    """Trainiert das LDA-Modell."""
    lda_model = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=42,
        passes=passes,
        iterations=iterations,
        eval_every=eval_every
    )
    return lda_model


def evaluate_model(lda_model, corpus, dictionary, processed_docs):
    """Berechnet den Coherence Score."""
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary,
                                          coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence)
    return coherence


def display_topics(lda_model, num_words=10):
    """Gibt die Top-Wörter pro Topic aus."""
    for idx, topic in lda_model.print_topics(-1, num_words=num_words):
        print(f'Topic: {idx} \nWords: {topic}')


def visualize_lda(lda_model, corpus, dictionary):
    """Erzeugt eine interaktive LDA-Visualisierung."""
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
    pyLDAvis.display(vis)


def remove_top_words(processed_docs, top_words):
    """Entfernt die häufigsten Wörter aus den Dokumenten."""
    processed_docs = [[word for word in doc if word not in top_words] for doc in processed_docs]
    return processed_docs


def main():
    """Hauptfunktion zum Ausführen der LDA-Analyse."""
    # Daten laden
    df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')
    review_column = 'rezension'
    rating_column = 'bewertung'

    # Fehlermeldung, wenn die Spalte nicht gefunden wird
    if review_column not in df.columns or rating_column not in df.columns:
        raise ValueError(f"Column '{review_column}' or '{rating_column}' not found in DataFrame.")

    # Filtern der Daten nach positiven und negativen Bewertungen
    positive_df = df[df[rating_column] >= 4]
    negative_df = df[df[rating_column] <= 2]

    # LDA für positive Bewertungen
    print("Starte LDA für positive Bewertungen:")
    positive_reviews = positive_df[review_column].astype(str)
    processed_positive = positive_reviews.map(preprocess_text)
    dictionary_positive, corpus_positive = prepare_corpus(processed_positive)

    # Ermitteln der optimalen Anzahl an Topics (optional)
    # Um eine optimale Anzahl an Topics zu ermitteln, kann man den Coherence Score für verschiedene
    # Topic-Anzahlen berechnen und diejenige mit dem höchsten Score auswählen.

    num_topics_positive = 5
    lda_model_positive = train_lda(corpus_positive, dictionary_positive, num_topics=num_topics_positive)
    coherence_score_positive = evaluate_model(lda_model_positive, corpus_positive, dictionary_positive,
                                               processed_positive)
    display_topics(lda_model_positive)
    visualize_lda(lda_model_positive, corpus_positive, dictionary_positive)

    # Manuelles Entfernen von Wörtern
    words_to_remove = input("Geben Sie Wörter an, die aus den Topics entfernt werden sollen (getrennt durch Komma): ").split(',')
    words_to_remove = [word.strip() for word in words_to_remove]  # Leerzeichen entfernen
    processed_positive = remove_top_words(processed_positive, words_to_remove)

    # Erneutes Trainieren des LDA-Modells mit bereinigten Daten
    dictionary_positive, corpus_positive = prepare_corpus(processed_positive)
    lda_model_positive = train_lda(corpus_positive, dictionary_positive, num_topics=num_topics_positive)
    coherence_score_positive = evaluate_model(lda_model_positive, corpus_positive, dictionary_positive,
                                               processed_positive)
    display_topics(lda_model_positive)
    visualize_lda(lda_model_positive, corpus_positive, dictionary_positive)

    # LDA für negative Bewertungen
    print("\nStarte LDA für negative Bewertungen:")
    negative_reviews = negative_df[review_column].astype(str)
    processed_negative = negative_reviews.map(preprocess_text)
    dictionary_negative, corpus_negative = prepare_corpus(processed_negative)

    num_topics_negative = 5
    lda_model_negative = train_lda(corpus_negative, dictionary_negative, num_topics=num_topics_negative)
    coherence_score_negative = evaluate_model(lda_model_negative, corpus_negative, dictionary_negative,
                                               processed_negative)
    display_topics(lda_model_negative)
    visualize_lda(lda_model_negative, corpus_negative, dictionary_negative)

    # Manuelles Entfernen von Wörtern
    words_to_remove = input("Geben Sie Wörter an, die aus den Topics entfernt werden sollen (getrennt durch Komma): ").split(',')
    words_to_remove = [word.strip() for word in words_to_remove]  # Leerzeichen entfernen
    processed_negative = remove_top_words(processed_negative, words_to_remove)

    # Erneutes Trainieren des LDA-Modells mit bereinigten Daten
    dictionary_negative, corpus_negative = prepare_corpus(processed_negative)
    lda_model_negative = train_lda(corpus_negative, dictionary_negative, num_topics=num_topics_negative)
    coherence_score_negative = evaluate_model(lda_model_negative, corpus_negative, dictionary_negative,
                                               processed_negative)
    display_topics(lda_model_negative)
    visualize_lda(lda_model_negative, corpus_negative, dictionary_negative)


if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Starte LDA für positive Bewertungen:

Coherence Score:  0.5356109172096308
Topic: 0 
Words: 0.019*"gut" + 0.017*"wurde" + 0.014*"freundlich" + 0.011*"station" + 0.011*"krankenhaus" + 0.011*"super" + 0.010*"ärzte" + 0.009*"dank" + 0.009*"personal" + 0.009*"immer"
Topic: 1 
Words: 0.010*"nan" + 0.002*"the" + 0.002*"and" + 0.001*"bus" + 0.001*"freie" + 0.001*"moegelin" + 0.001*"fürsorglichen" + 0.001*"weiterbehandlung" + 0.001*"passen" + 0.001*"vati"
Topic: 2 
Words: 0.016*"team" + 0.015*"klinik" + 0.012*"dank" + 0.008*"zeit" + 0.008*"gut" + 0.007*"möchte" + 0.007*"vielen" + 0.007*"frau" + 0.006*"gesamte" + 0.006*"betreuung"
Topic: 3 
Words: 0.014*"klinik" + 0.010*"immer" + 0.010*"gut" + 0.010*"essen" + 0.009*"gibt" + 0.007*"patienten" + 0.007*"therapeuten" + 0.007*"zimmer" + 0.006*"wochen" + 0.006*"mal"
Topic: 4 
Words: 0.019*"hebammen" + 0.019*"geburt" + 0.010*"welt" + 0.008*"kind" + 0.008*"kreißsaal" + 0.008*"hebamme" + 0.005*"entbunden" + 0.005*"tochter" + 0.004*"kaiserschnitt" + 0.00

Überarbeitung 2

In [4]:
nltk.download('stopwords')

# Load data
df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')
review_column = 'rezension'
if review_column not in df.columns:
    raise ValueError(f"Column '{review_column}' not found in DataFrame.")

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', str(text))
    text = text.lower()
    tokens = text.split()
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens

def prepare_corpus(processed_docs): #Gensim-dictionary und Gensim-Korpus
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    return dictionary, corpus

# n = k, festlegen!! Iteratives Trainieren des LDA Modells
def train_lda(corpus, dictionary, num_topics=5, passes=15, iterations=400, eval_every=None):
    lda_model = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=42,
        passes=passes,
        iterations=iterations,
        eval_every=eval_every
    )
    return lda_model

# Coherence-Score; je höher, desto 'bessere' Topics
def evaluate_model(lda_model, corpus, dictionary, processed_docs):
    """Berechnet den Coherence Score."""
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary,
                                          coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence)
    return coherence

# erste 10 topics anzeigen
def display_topics(lda_model, num_words=10):
    """Gibt die Top-Wörter pro Topic aus."""
    for idx, topic in lda_model.print_topics(-1, num_words=num_words):
        print(f'Topic: {idx} \nWords: {topic}')

# erste Visualisierung
def visualize_lda(lda_model, corpus, dictionary):
    """Erzeugt eine interaktive LDA-Visualisierung."""
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
    pyLDAvis.display(vis)



def main():
    """Hauptfunktion zum Ausführen der LDA-Analyse."""
    # Daten laden
    df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')
    review_column = 'rezension'
    rating_column = 'bewertung'

    # Fehlermeldung, wenn die Spalte nicht gefunden wird
    if review_column not in df.columns or rating_column not in df.columns:
        raise ValueError(f"Column '{review_column}' or '{rating_column}' not found in DataFrame.")

    # Filtern der Daten nach positiven und negativen Bewertungen
    positive_df = df[df[rating_column] >= 4]
    negative_df = df[df[rating_column] <= 2]


    # LDA für positive Bewertungen
    print("Starte LDA für positive Bewertungen:")
    positive_reviews = positive_df[review_column].astype(str)
    processed_positive = positive_reviews.map(preprocess_text)
    dictionary_positive, corpus_positive = prepare_corpus(processed_positive)

    num_topics_positive = 5
    lda_model_positive = train_lda(corpus_positive, dictionary_positive, num_topics=num_topics_positive)
    coherence_score_positive = evaluate_model(lda_model_positive, corpus_positive, dictionary_positive,
                                               processed_positive)
    display_topics(lda_model_positive)
    visualize_lda(lda_model_positive, corpus_positive, dictionary_positive)


    # LDA für negative Bewertungen
    print("\nStarte LDA für negative Bewertungen:")
    negative_reviews = negative_df[review_column].astype(str)
    processed_negative = negative_reviews.map(preprocess_text)
    dictionary_negative, corpus_negative = prepare_corpus(processed_negative)

    num_topics_negative = 5
    lda_model_negative = train_lda(corpus_negative, dictionary_negative, num_topics=num_topics_negative)
    coherence_score_negative = evaluate_model(lda_model_negative, corpus_negative, dictionary_negative,
                                               processed_negative)
    display_topics(lda_model_negative)
    visualize_lda(lda_model_negative, corpus_negative, dictionary_negative)


if __name__ == "__main__":
    main()


# Visualisierung mit pyLDAvis
vis_neg = pyLDAvis.gensim_models.prepare(lda_model_negative, corpus, dictionary) # prepare aus gensim_models
pyLDAvis.display(vis_neg)


# Visualisierung mit pyLDAvis
vis_pos = pyLDAvis.gensim_models.prepare(lda_model_positive, corpus, dictionary) # prepare aus gensim_models
pyLDAvis.display(vis_pos)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Starte LDA für positive Bewertungen:


NameError: name 'corpora' is not defined

In [6]:
nltk.download('stopwords')

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', str(text))
    text = text.lower()
    tokens = text.split()
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens

def prepare_corpus(processed_docs):
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    return dictionary, corpus

def train_lda(corpus, dictionary, num_topics=5, passes=15, iterations=400, eval_every=None):
    lda_model = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=42,
        passes=passes,
        iterations=iterations,
        eval_every=eval_every
    )
    return lda_model

def evaluate_model(lda_model, corpus, dictionary, processed_docs):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary,
                                          coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence)
    return coherence

def display_topics(lda_model, num_words=10):
    for idx, topic in lda_model.print_topics(-1, num_words=num_words):
        print(f'Topic: {idx} \nWords: {topic}')

def visualize_lda(lda_model, corpus, dictionary):
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
    pyLDAvis.display(vis)

def main():
    df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')
    review_column = 'rezension'
    rating_column = 'bewertung'

    if review_column not in df.columns or rating_column not in df.columns:
        raise ValueError(f"Column '{review_column}' or '{rating_column}' not found in DataFrame.")

    positive_df = df[df[rating_column] >= 4]
    negative_df = df[df[rating_column] <= 2]

    print("Starte LDA für positive Bewertungen:")
    positive_reviews = positive_df[review_column].astype(str)
    processed_positive = positive_reviews.map(preprocess_text)
    dictionary_positive, corpus_positive = prepare_corpus(processed_positive)

    num_topics_positive = 5
    lda_model_positive = train_lda(corpus_positive, dictionary_positive, num_topics=num_topics_positive)
    coherence_score_positive = evaluate_model(lda_model_positive, corpus_positive, dictionary_positive,
                                               processed_positive)
    display_topics(lda_model_positive)
    visualize_lda(lda_model_positive, corpus_positive, dictionary_positive)

    print("\nStarte LDA für negative Bewertungen:")
    negative_reviews = negative_df[review_column].astype(str)
    processed_negative = negative_reviews.map(preprocess_text)
    dictionary_negative, corpus_negative = prepare_corpus(processed_negative)

    num_topics_negative = 5
    lda_model_negative = train_lda(corpus_negative, dictionary_negative, num_topics=num_topics_negative)
    coherence_score_negative = evaluate_model(lda_model_negative, corpus_negative, dictionary_negative,
                                               processed_negative)
    display_topics(lda_model_negative)
    visualize_lda(lda_model_negative, corpus_negative, dictionary_negative)

    # Visualisierung mit pyLDAvis for negative reviews
    vis_neg = pyLDAvis.gensim_models.prepare(lda_model_negative, corpus_negative, dictionary_negative)
    pyLDAvis.display(vis_neg)

    # Visualisierung mit pyLDAvis for positive reviews
    vis_pos = pyLDAvis.gensim_models.prepare(lda_model_positive, corpus_positive, dictionary_positive) # prepare aus gensim_models
    pyLDAvis.display(vis_pos)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Starte LDA für positive Bewertungen:

Coherence Score:  0.5356109172096308
Topic: 0 
Words: 0.019*"gut" + 0.017*"wurde" + 0.014*"freundlich" + 0.011*"station" + 0.011*"krankenhaus" + 0.011*"super" + 0.010*"ärzte" + 0.009*"dank" + 0.009*"personal" + 0.009*"immer"
Topic: 1 
Words: 0.010*"nan" + 0.002*"the" + 0.002*"and" + 0.001*"bus" + 0.001*"freie" + 0.001*"moegelin" + 0.001*"fürsorglichen" + 0.001*"weiterbehandlung" + 0.001*"passen" + 0.001*"vati"
Topic: 2 
Words: 0.016*"team" + 0.015*"klinik" + 0.012*"dank" + 0.008*"zeit" + 0.008*"gut" + 0.007*"möchte" + 0.007*"vielen" + 0.007*"frau" + 0.006*"gesamte" + 0.006*"betreuung"
Topic: 3 
Words: 0.014*"klinik" + 0.010*"immer" + 0.010*"gut" + 0.010*"essen" + 0.009*"gibt" + 0.007*"patienten" + 0.007*"therapeuten" + 0.007*"zimmer" + 0.006*"wochen" + 0.006*"mal"
Topic: 4 
Words: 0.019*"hebammen" + 0.019*"geburt" + 0.010*"welt" + 0.008*"kind" + 0.008*"kreißsaal" + 0.008*"hebamme" + 0.005*"entbunden" + 0.005*"tochter" + 0.004*"kaiserschnitt" + 0.00

In [7]:
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models
import pyLDAvis

nltk.download('stopwords')

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', str(text))
    text = text.lower()
    tokens = text.split()
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    return tokens

def prepare_corpus(processed_docs):
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    return dictionary, corpus

def train_lda(corpus, dictionary, num_topics=5):
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42)
    return lda_model

def evaluate_model(lda_model, corpus, dictionary, processed_docs):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence)
    return coherence

def display_topics(lda_model, num_words=10):
    for idx, topic in lda_model.print_topics(-1, num_words=num_words):
        print(f'Topic: {idx} \nWords: {topic}')

def visualize_lda(lda_model, corpus, dictionary):
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
    pyLDAvis.display(vis)

def main():
    df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')
    review_column = 'rezension'
    rating_column = 'bewertung'

    if review_column not in df.columns or rating_column not in df.columns:
        raise ValueError(f"Column '{review_column}' or '{rating_column}' not found in DataFrame.")

    positive_df = df[df[rating_column] >= 4]
    negative_df = df[df[rating_column] <= 2]

    print("Starte LDA für positive Bewertungen:")
    positive_reviews = positive_df[review_column].astype(str)
    processed_positive = positive_reviews.map(preprocess_text)
    dictionary_positive, corpus_positive = prepare_corpus(processed_positive)

    num_topics_positive = 5
    lda_model_positive = train_lda(corpus_positive, dictionary_positive, num_topics=num_topics_positive)
    coherence_score_positive = evaluate_model(lda_model_positive, corpus_positive, dictionary_positive,processed_positive)
    display_topics(lda_model_positive)
    visualize_lda(lda_model_positive, corpus_positive, dictionary_positive)

    print("\nStarte LDA für negative Bewertungen:")
    negative_reviews = negative_df[review_column].astype(str)
    processed_negative = negative_reviews.map(preprocess_text)
    dictionary_negative, corpus_negative = prepare_corpus(processed_negative)

    num_topics_negative = 5
    lda_model_negative = train_lda(corpus_negative, dictionary_negative, num_topics=num_topics_negative)
    coherence_score_negative = evaluate_model(lda_model_negative, corpus_negative, dictionary_negative,processed_negative)
    display_topics(lda_model_negative)
    visualize_lda(lda_model_negative, corpus_negative, dictionary_negative)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Starte LDA für positive Bewertungen:



Coherence Score:  0.40011651397111986
Topic: 0 
Words: 0.019*"gut" + 0.016*"wurde" + 0.012*"freundlich" + 0.011*"station" + 0.010*"dank" + 0.009*"krankenhaus" + 0.009*"ärzte" + 0.008*"personal" + 0.008*"klinik" + 0.008*"super"
Topic: 1 
Words: 0.007*"nan" + 0.006*"tagesklinik" + 0.005*"gut" + 0.005*"klinik" + 0.004*"super" + 0.004*"patienten" + 0.004*"zeit" + 0.004*"immer" + 0.004*"freundlich" + 0.004*"danke"
Topic: 2 
Words: 0.020*"klinik" + 0.012*"team" + 0.011*"gut" + 0.011*"wurde" + 0.008*"dank" + 0.007*"freundlich" + 0.006*"zeit" + 0.006*"danke" + 0.006*"vielen" + 0.005*"ärzte"
Topic: 3 
Words: 0.014*"immer" + 0.011*"klinik" + 0.008*"gut" + 0.008*"essen" + 0.008*"gibt" + 0.007*"patienten" + 0.006*"freundlich" + 0.005*"zeit" + 0.005*"zimmer" + 0.005*"mal"
Topic: 4 
Words: 0.009*"dank" + 0.009*"möchte" + 0.007*"betreuung" + 0.006*"zeit" + 0.006*"geburt" + 0.006*"frau" + 0.005*"team" + 0.005*"gut" + 0.005*"vielen" + 0.005*"patienten"

Starte LDA für negative Bewertungen:



Coherence Score:  0.2859322586973052
Topic: 0 
Words: 0.016*"wurde" + 0.007*"leider" + 0.006*"mehr" + 0.006*"klinik" + 0.005*"mal" + 0.005*"stunden" + 0.004*"einfach" + 0.004*"krankenhaus" + 0.004*"station" + 0.003*"tag"
Topic: 1 
Words: 0.015*"patienten" + 0.010*"klinik" + 0.009*"wurde" + 0.006*"arzt" + 0.005*"mehr" + 0.005*"mal" + 0.005*"einfach" + 0.005*"menschen" + 0.005*"ärzte" + 0.004*"krankenhaus"
Topic: 2 
Words: 0.012*"wurde" + 0.008*"krankenhaus" + 0.006*"mehr" + 0.006*"termin" + 0.005*"patienten" + 0.004*"uhr" + 0.004*"stunden" + 0.004*"warten" + 0.004*"notaufnahme" + 0.004*"arzt"
Topic: 3 
Words: 0.015*"wurde" + 0.009*"klinik" + 0.008*"mal" + 0.006*"notaufnahme" + 0.005*"leider" + 0.005*"mutter" + 0.005*"mehr" + 0.005*"patienten" + 0.005*"schon" + 0.004*"arzt"
Topic: 4 
Words: 0.016*"wurde" + 0.009*"krankenhaus" + 0.006*"station" + 0.005*"patienten" + 0.005*"notaufnahme" + 0.005*"einfach" + 0.004*"arzt" + 0.004*"mal" + 0.004*"tag" + 0.004*"mehr"
